## Scraping Børsen for articles about Novo Nordisk

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
import os
from bs4 import BeautifulSoup
import sys
sys.path.append('/Users/rosemarieblicher/Desktop/kode_koner/PROJECT/Børsen')  # replace with your actual path
from Credentials import *

In [ ]:
# Save for later use
response = requests.get('https://borsen.dk/', headers={'name': 'Rosemarie Thalia Blicher', 'e-mail': 'mzc144@alumni.ku.dk'})

# Set up Selenium with Chrome driver
driver = webdriver.Chrome()
driver.maximize_window()

# Specify the URL
url = "https://borsen.dk/"
driver.get(url)
time.sleep(0.5)  # Add a delay to avoid overwhelming the server and getting blocked

# We want the cookies to go away
cookie = driver.find_element(By.ID, 'acceptButton')
cookie.click()
time.sleep(0.5) # Cookie pop-up will disappear

# Click on the search button
search_button = driver.find_element(By.CLASS_NAME, 'icon-search')
search_button.click()

# Wait for the search input field to appear (you might need to adjust the waiting time)
import time
time.sleep(2)  # Wait for 2 seconds

# Find the search input field and type in "Novo Nordisk"
search_input = driver.find_element(By.XPATH, '//input[@type="text"]')
search_input.send_keys('Novo Nordisk')
search_input.send_keys(Keys.RETURN)

In [ ]:
# Now we want to log in, such that we get access to all articles
login_button = driver.find_element(By.XPATH, "//span[text()='Log ind']")
login_button.click()

# Find the email input field by its id and type in the email
email_input = driver.find_element(By.ID, "__BVID__214")
email_input.send_keys("fzb@an.dk")

# Find the password input field by its id and type in the password
password_input = driver.find_element(By.ID, "__BVID__216")
password_input.send_keys(Password)